# OpenAI API 강의 5
## 주제: 파인튜닝(Fine-Tuning)과 커스텀 모델 제작


## 학습 목표
- OpenAI Fine-Tuning의 개념 이해
- JSONL 형식의 학습 데이터 준비 방법 학습
- fine_tuning.jobs.create() 활용 실습
- 커스텀 모델을 생성하고 테스트하는 방법 익히기



## 1. 파인튜닝(Fine-Tuning)이란?
| 항목 | 설명 |
|------|------|
| 정의 | 기존 사전학습(Pretrained) 모델을 특정 데이터로 추가 학습시키는 과정 |
| 목적 | 특정 도메인(예: 금융, 법률, 기술)에 특화된 모델 제작 |
| 적용 예시 | 기업 Q&A 챗봇, 문서 요약, 스타일 고정 대화형 모델 |



## 2. 데이터 형식 준비
파인튜닝 학습 데이터는 반드시 **JSONL (JSON Lines)** 형식이어야 합니다.
각 줄은 하나의 학습 예시로 구성됩니다.

예시:
```json
{"messages": [{"role": "system", "content": "너는 금융 전문가야."}, {"role": "user", "content": "ETF란 무엇인가?"}, {"role": "assistant", "content": "ETF는 상장지수펀드로..."}]}
{"messages": [{"role": "system", "content": "너는 금융 전문가야."}, {"role": "user", "content": "채권의 이자율이 오르면 무슨 일이 생기나요?"}, {"role": "assistant", "content": "채권 가격이 하락합니다."}]}
```


In [ ]:
# 예시 JSONL 파일 생성
import json

data = [
    {"messages": [
        {"role": "system", "content": "너는 금융 전문가야."},
        {"role": "user", "content": "ETF란 무엇인가?"},
        {"role": "assistant", "content": "ETF는 상장지수펀드로, 여러 자산을 묶어 거래소에 상장한 투자 상품입니다."}
    ]},
    {"messages": [
        {"role": "system", "content": "너는 금융 전문가야."},
        {"role": "user", "content": "채권 금리가 오르면 어떤 일이 생기나요?"},
        {"role": "assistant", "content": "채권 가격이 하락하게 됩니다."}
    ]}
]

with open("finance_finetune.jsonl", "w", encoding="utf-8") as f:
    for item in data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print("finance_finetune.jsonl 파일이 생성되었습니다.")


finance_finetune.jsonl 파일이 생성되었습니다.



## 3. 파인튜닝 작업 생성하기

다음 코드는 준비한 JSONL 파일을 업로드하고, 파인튜닝 작업을 생성하는 예시입니다.


In [3]:
from openai import OpenAI
client = OpenAI()

# 1) 학습 파일 업로드
file = client.files.create(
    file=open("finance_finetune.jsonl", "rb"),
    purpose="fine-tune"
)
print("업로드 완료:", file.id)

# 2) 파인튜닝 작업 생성
fine_tune = client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"   # 파인튜닝 가능 모델 확인 필요
)

print("파인튜닝 작업 생성:", fine_tune.id)


업로드 완료: file-Wg6FEjUgkMRk4w6droiLHj
파인튜닝 작업 생성: ftjob-yUtogckmYgTfmUZzRDtkFmDB



## 4. 파인튜닝 상태 모니터링
파인튜닝이 완료되면 커스텀 모델 이름이 생성됩니다.


In [ ]:
# 파인튜닝 상태 확인
status = client.fine_tuning.jobs.retrieve(fine_tune.id)
print(status)


FineTuningJob(id='ftjob-yUtogckmYgTfmUZzRDtkFmDB', created_at=1760058046, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-DdAJafyw8vIZRDzL2s6Acpcq', result_files=[], seed=592568287, status='validating_files', trained_tokens=None, training_file='file-Wg6FEjUgkMRk4w6droiLHj', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)


In [6]:
# 최근 파인튜닝 작업 목록 가져오기 (최대 10개)
fine_tuning_jobs = client.fine_tuning.jobs.list(limit=10)

# 각 작업의 상태와 생성된 모델 ID 출력
for job in fine_tuning_jobs.data:
    print(f"Job ID: {job.id}")
    print(f"Status: {job.status}")
    print(f"Fine-tuned Model: {job.fine_tuned_model}") # <--- 이 값을 사용해야 합니다!
    print("-" * 20)


Job ID: ftjob-yUtogckmYgTfmUZzRDtkFmDB
Status: failed
Fine-tuned Model: None
--------------------



## 5. 커스텀 모델 사용하기
파인튜닝이 완료되면 새 모델을 사용할 수 있습니다.


In [5]:
custom_model = "ft:gpt-4o-mini:finance-expert:2025-10-10"

response = client.chat.completions.create(
    model=custom_model,
    messages=[
        {"role": "user", "content": "ETF와 펀드의 차이를 설명해줘."}
    ]
)

print(response.choices[0].message.content)


NotFoundError: Error code: 404 - {'error': {'message': 'The model `ft:gpt-4o-mini:finance-expert:2025-10-10` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

-----

### **OpenAI 파인튜닝 - Jupyter Notebook 실습**

이 노트북은 다음의 5단계로 구성되어 있습니다. 각 코드 셀을 순서대로 실행하며 파인튜닝의 전체 과정을 학습합니다.

1.  **데이터 준비**: 파인튜닝에 사용할 훈련 데이터를 `JSONL` 형식으로 생성합니다.
2.  **파일 업로드**: 생성한 훈련 파일을 OpenAI 서버에 업로드합니다.
3.  **파인튜닝 작업 생성**: 업로드한 파일로 파인튜닝 작업을 시작합니다.
4.  **작업 상태 모니터링**: 파인튜닝 작업이 완료될 때까지 상태를 확인합니다.
5.  **모델 사용**: 완성된 커스텀 모델을 이용해 테스트를 진행합니다.

-----

### **1단계: 훈련 데이터 준비**

#### **셀 1: 필요 라이브러리 임포트 및 클라이언트 설정**

가장 먼저 필요한 라이브러리를 가져오고 OpenAI 클라이언트를 초기화합니다.



In [22]:
import os
import json
import time
from openai import OpenAI

# OpenAI 클라이언트 초기화 (API 키는 환경변수에서 가져옵니다)
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY" # 필요한 경우 주석을 해제하고 키를 입력하세요.
client = OpenAI()


#### **셀 2: 훈련 데이터 정의 및 JSONL 파일 생성**

'현명한 학자' 말투를 학습시키기 위한 대화 데이터를 정의하고, 이를 `jsonl` 파일 형식으로 저장합니다.


In [30]:
import json

# 훈련 데이터 파일명 정의
TRAINING_FILE_NAME = "training_data_10.jsonl"

# 10개의 대화 예제로 확장된 훈련 데이터
training_data = [
    # --- 기존 예제 3개 ---
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "ETF란 무엇인가?"},
            {"role": "assistant", "content": "음, 좋은 질문이오. ETF란 '거래소 거래 펀드'의 약자로, 주식처럼 거래소에서 실시간으로 사고팔 수 있는 펀드를 일컫는 말이오."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "인공지능의 미래에 대해 어떻게 생각하시오?"},
            {"role": "assistant", "content": "허허, 인공지능의 미래라... 기술의 발전은 양날의 검과 같아서, 인류에게 큰 번영을 가져다줄 수도 있지만, 그에 따르는 윤리적 고찰도 함께 이루어져야 할 것이오."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "양자 컴퓨팅에 대해 알려주시오."},
            {"role": "assistant", "content": "양자 컴퓨팅이라 함은, 양자역학의 원리를 이용하여 기존 컴퓨터의 한계를 뛰어넘는 새로운 계산 방식을 말하는 것이오. 중첩과 얽힘 같은 특성을 활용하지."}
        ]
    },
    # --- 새로운 예제 7개 추가 ---
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "블록체인 기술에 대해 쉽게 설명해주시오."},
            {"role": "assistant", "content": "블록체인이란 말이오, 거래 기록을 담은 '블록'들을 '사슬'처럼 연결한 장부라 할 수 있소. 중앙 관리자 없이도 모든 참여자가 장부를 공유하여 위변조가 거의 불가능한 것이 특징이지."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "인플레이션은 왜 발생하는 것이오?"},
            {"role": "assistant", "content": "허허, 경제의 순리는 참으로 오묘하지. 인플레이션은 시중에 통화량이 많아져 돈의 가치가 떨어지거나, 물건에 대한 수요가 공급을 크게 초과할 때 발생하는 현상이오."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "블랙홀이란 무엇인지 궁금하오."},
            {"role": "assistant", "content": "우주의 심연을 논하는구려. 블랙홀은 중력이 너무나도 강력하여 빛조차 빠져나올 수 없는 시공간의 한 영역을 일컫는 말이오. 모든 것을 빨아들이는 끝없는 구멍과도 같지."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "스토아 철학의 핵심은 무엇이오?"},
            {"role": "assistant", "content": "스토아 철학의 정수는 '우리가 통제할 수 있는 것과 없는 것을 구분하는 지혜'에 있소. 외적 사건에 흔들리지 않고 내면의 평온과 덕을 추구하는 것이 핵심이지."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "커피에 대해 어찌 생각하시오?"},
            {"role": "assistant", "content": "그 검고 향기로운 물 말이오? 적절히 마시면 정신을 맑게 하는 좋은 벗이 되지만, 과하면 마음의 평온을 해칠 수 있으니 중용의 도를 지키는 것이 중요하오."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "소셜 미디어에 대한 학자님의 견해는?"},
            {"role": "assistant", "content": "허허, 세상의 모든 소식이 손안에 오가는 편리한 장이 열렸으나, 그만큼이나 헛된 명예와 시기심이 자라나는 토양이 되기도 하니, 현명하게 가려 쓸 지혜가 필요하다 할 것이오."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": "요즘 유행하는 메타버스란 무엇이오?"},
            {"role": "assistant", "content": "메타버스란 가상의 세계에서 사람들이 교류하며 현실과 같은 활동을 하는 공간을 일컫는 말이오. 기술이 만들어낸 또 다른 세상의 그림자라 할 수 있겠지."}
        ]
    }
]

# JSONL 파일로 데이터 쓰기
with open(TRAINING_FILE_NAME, "w", encoding="utf-8") as f:
    for entry in training_data:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

print(f"총 10개의 예제를 포함한 훈련 데이터 파일 '{TRAINING_FILE_NAME}'이(가) 성공적으로 생성되었습니다.")

총 10개의 예제를 포함한 훈련 데이터 파일 'training_data_10.jsonl'이(가) 성공적으로 생성되었습니다.


### **2단계: 훈련 파일 업로드**

#### **셀 3: 생성한 훈련 파일을 OpenAI에 업로드**

방금 생성한 `training_data.jsonl` 파일을 OpenAI의 `fine-tune` 목적으로 업로드합니다. 업로드가 완료되면 다음 단계에서 사용할 **File ID**가 출력됩니다.



In [31]:
# 파일 열어서 업로드
with open(TRAINING_FILE_NAME, "rb") as f:
    training_file = client.files.create(file=f, purpose="fine-tune")

print(f"파일 업로드 완료. File ID: {training_file.id}")


파일 업로드 완료. File ID: file-HkwUd3cki5MhdmmbN6LC72


### **3단계: 파인튜닝 작업 생성**

#### **셀 4: 업로드한 파일을 이용하여 파인튜닝 작업 시작**

앞 단계에서 얻은 **File ID**를 사용하여 파인튜닝 작업을 시작합니다. 여기서는 기본 모델로 `gpt-3.5-turbo-0125`를 사용합니다. 작업이 생성되면 **Job ID**가 출력됩니다.


In [32]:
# 사용할 기본 모델 정의
BASE_MODEL = "gpt-3.5-turbo-0125"

# 파인튜닝 작업 생성
job = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    model=BASE_MODEL
)

# 생성된 Job ID 저장
job_id = job.id

print(f"파인튜닝 작업 생성 완료. Job ID: {job_id}")


파인튜닝 작업 생성 완료. Job ID: ftjob-6NqzJxxcWEwMLw18m4wmUDQh



### **4단계: 파인튜닝 작업 상태 모니터링**

#### **셀 5: 작업 상태 확인**

파인튜닝은 완료까지 수 분에서 수 시간까지 소요될 수 있습니다. 아래 셀을 **주기적으로 실행**하여 작업 상태를 확인하세요. 상태가 `running`에서 \*\*`succeeded`\*\*로 바뀌면 다음 단계로 진행할 수 있습니다.


In [38]:
# Job ID를 사용하여 현재 작업 상태 검색
job_status = client.fine_tuning.jobs.retrieve(job_id)

print(f"현재 작업 상태: {job_status.status}")

현재 작업 상태: running


In [36]:
job_status

FineTuningJob(id='ftjob-6NqzJxxcWEwMLw18m4wmUDQh', created_at=1760059225, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-DdAJafyw8vIZRDzL2s6Acpcq', result_files=[], seed=1366259860, status='running', trained_tokens=None, training_file='file-HkwUd3cki5MhdmmbN6LC72', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)


#### **셀 6: (선택) 작업 완료 후, 생성된 모델 ID 확인**

작업 상태가 \*\*`succeeded`\*\*인 것을 확인한 후에 이 셀을 실행하세요. 파인튜닝을 통해 최종적으로 생성된 커스텀 모델의 ID를 확인할 수 있습니다.


In [37]:
# 작업이 성공했을 때만 실행
job_details = client.fine_tuning.jobs.retrieve(job_id)

if job_details.status == "succeeded":
    fine_tuned_model_id = job_details.fine_tuned_model
    print(f"파인튜닝 성공! 생성된 모델 ID: {fine_tuned_model_id}")
else:
    print(f"작업이 아직 완료되지 않았습니다. 현재 상태: {job_details.status}")


작업이 아직 완료되지 않았습니다. 현재 상태: running


### **5단계: 파인튜닝된 모델 사용하기**

#### **셀 7: 완성된 커스텀 모델로 대화 테스트**

이제 `fine_tuned_model_id` 변수에 저장된 ID를 사용하여 우리만의 커스텀 모델에 질문을 하고, 학습된 '현명한 학자'의 말투로 답변하는지 확인합니다.


In [21]:
# 파인튜닝된 모델 ID가 있는지 확인 후 실행
if 'fine_tuned_model_id' in locals() and fine_tuned_model_id:
    test_prompt = "요즘 유행하는 '밈(Meme)'에 대해 설명해주시오."
    print(f"질문: {test_prompt}\n")

    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=[
            {"role": "system", "content": "당신은 세상의 모든 지식을 꿰뚫고 있는 현명한 학자입니다. 항상 학자풍의 고상한 말투를 사용해야 합니다."},
            {"role": "user", "content": test_prompt}
        ]
    )
    
    assistant_reply = response.choices[0].message.content
    print(f"학자의 답변: {assistant_reply}")
else:
    print("모델 ID가 아직 준비되지 않았습니다. 4단계를 완료했는지 확인하세요.")


모델 ID가 아직 준비되지 않았습니다. 4단계를 완료했는지 확인하세요.



## 6. 실습 과제
1. 자신이 관심 있는 주제(예: 교육, 여행, 과학 등)로 JSONL 데이터를 만들어 보세요.
2. 파인튜닝 작업을 실제로 생성하고 상태를 확인해 보세요.
3. 완성된 모델을 테스트하고 기존 모델과의 응답 차이를 비교해 보세요.



## 참고 자료
- [OpenAI Fine-tuning Guide](https://platform.openai.com/docs/guides/fine-tuning)
- [Fine-tuning Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/Fine_tuning_Guide.ipynb)
- [JSONL 포맷 설명](https://jsonlines.org/)
